## Building Tools for LLM 🛠️


In [16]:
from dotenv import load_dotenv
load = load_dotenv('./../.env')

In [17]:
from dotenv import load_dotenv
from langchain_ollama import ChatOllama

load = load_dotenv('./../.env')


llm = ChatOllama(
    base_url="http://localhost:11434",
    model = "qwen2.5:3b",
    temperature=0.5,
    max_tokens = 250
)

### Installing a community

In [18]:
%pip install --upgrade --quiet  wikipedia

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [19]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [20]:
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

tool_response = wikipedia.invoke("Give me details of Avatar movie")

print(tool_response)

Page: Avatar (franchise)
Summary: Avatar is an American epic science fiction media franchise created by James Cameron, which began with the eponymous 2009 film. Produced by 20th Century Studios and distributed by Lightstorm Entertainment, it consists of associated merchandise, video games, and theme park attractions. Avatar is set in the mid-22nd century on Pandora, a lush habitable moon of a gas giant in the Alpha Centauri star system. The films' central conflict is between the indigenous Na'vi led by Jake Sully and Neytiri, and humans led by Colonel Miles Quaritch from the Resources Development Administration (RDA), a megacorp which has arrived on Pandora to colonize and pillage it for its natural resources. The title of the series refers to the genetically engineered Na'vi body operated from the brain that humans pilot to interact with on Pandora.
The first installment, Avatar, was released on December 18, 2009, and is the highest grossing motion picture of all-time when ticket pric

### Creating Custom Tools 

In [21]:
from langchain.tools import tool

@tool
def add_numbers(a: int, b: int) -> int:
    "Add two numbers and return results."
    return int(a) + int(b)

@tool
def subtract_numbers(a: int, b: int) -> int:
    "Subtract two numbers and return results."
    return int(a) - int(b)

@tool
def multiply_numbers(a: int, b: int) -> int:
    "Multiply two numbers and return results."
    return int(a) * int(b)

print(add_numbers.invoke({"a": 10, "b": 20}))

30


In [22]:
tools = [wikipedia, add_numbers, subtract_numbers, multiply_numbers]

list_of_tools = {tool.name: tool for tool in tools}

list_of_tools

# response = llm.invoke("When did avatar movie released the ways of water?")

# print(response.content)

llm_with_tools = llm.bind_tools(tools=tools)

tool_call_response = llm_with_tools.invoke("What is the double of 2")

print(tool_call_response.tool_calls)

#tool_call_response


[{'name': 'multiply_numbers', 'args': {'a': 2, 'b': 2}, 'id': '2a71e6ba-8006-4885-ab33-88844b443975', 'type': 'tool_call'}]


### Execute the custom tools from LLM

In [40]:
from langchain_core.messages import HumanMessage

query = "Who is the tallest man in the world?"

message = []

message = [HumanMessage(query)]
ai_message = llm_with_tools.invoke(query)

message.append(ai_message)

message

[HumanMessage(content='Who is the tallest man in the world?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen2.5:3b', 'created_at': '2025-04-15T09:14:37.0917871Z', 'done': True, 'done_reason': 'stop', 'total_duration': 578594700, 'load_duration': 14218900, 'prompt_eval_count': 373, 'prompt_eval_duration': 206156900, 'eval_count': 41, 'eval_duration': 357140000, 'message': Message(role='assistant', content='', images=None, tool_calls=None), 'model_name': 'qwen2.5:3b'}, id='run-be5a4d96-3aaa-4c73-9a6d-8f61dca1a4ab-0', tool_calls=[{'name': 'wikipedia', 'args': {'query': 'tallest man in the world'}, 'id': '7cd68ea7-9b65-4d85-9b0f-0bba40b59de9', 'type': 'tool_call'}], usage_metadata={'input_tokens': 373, 'output_tokens': 41, 'total_tokens': 414})]

In [41]:
from langchain_core.messages import ToolMessage # Import ToolMessage

# Make sure message list is correctly initialized before this loop
# message = [HumanMessage(query), ai_message] # Should contain the human query and the AI's response with tool_calls

for tool_call in ai_message.tool_calls:
    tool_name = tool_call['name'].lower()
    
    # Check if the tool exists
    if tool_name in list_of_tools:
        execute_tool = list_of_tools[tool_name]
        
        # Extract the arguments dictionary
        tool_args = tool_call['args'] 
        
        try:
            # Invoke the tool with ONLY the arguments
            tool_output = execute_tool.invoke(tool_args) 
            
            # Append the result as a ToolMessage
            message.append(
                ToolMessage(tool_name = tool_name, content=str(tool_output), tool_args = tool_args,  tool_call_id=tool_call['id'])
            )
        except Exception as e:
            print(f"Error executing tool {tool_name} with args {tool_args}: {e}")
            # Optionally append an error message back
            message.append(
                ToolMessage(content=f"Error executing tool: {e}", tool_call_id=tool_call['id'])
            )
    else:
        print(f"Tool '{tool_name}' not found.")
        # Optionally append a message indicating the tool wasn't found
        message.append(
            ToolMessage(content=f"Tool '{tool_name}' not found.", tool_call_id=tool_call['id'])
        )

# Now 'message' contains the original query, the AI's request to call the tool, 
# and the ToolMessage with the result (or error) from the tool execution.
# You can then pass this updated 'message' list to the LLM again.
# final_output = llm_with_tools.invoke(message)
# print(final_output.content)
message

[HumanMessage(content='Who is the tallest man in the world?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen2.5:3b', 'created_at': '2025-04-15T09:14:37.0917871Z', 'done': True, 'done_reason': 'stop', 'total_duration': 578594700, 'load_duration': 14218900, 'prompt_eval_count': 373, 'prompt_eval_duration': 206156900, 'eval_count': 41, 'eval_duration': 357140000, 'message': Message(role='assistant', content='', images=None, tool_calls=None), 'model_name': 'qwen2.5:3b'}, id='run-be5a4d96-3aaa-4c73-9a6d-8f61dca1a4ab-0', tool_calls=[{'name': 'wikipedia', 'args': {'query': 'tallest man in the world'}, 'id': '7cd68ea7-9b65-4d85-9b0f-0bba40b59de9', 'type': 'tool_call'}], usage_metadata={'input_tokens': 373, 'output_tokens': 41, 'total_tokens': 414}),
 ToolMessage(content='Page: List of tallest people\nSummary: This is a list of the tallest people, verified by Guinness World Records or other reliable sources.\nAccording

In [42]:
# for tool_call in ai_message.tool_calls:
#     tool_name = tool_call['name'].lower()
    
#     execute_tool = list_of_tools[tool_name]
    
#     tool_invoke = execute_tool.invoke(tool_call)
    
#     message.append(tool_invoke)
    
# message

In [43]:
final_output = llm_with_tools.invoke(message)

print(final_output.content)

According to Guinness World Records, Robert Wadlow from the United States was the tallest person in recorded history, measuring 272 cm (8 feet 11 inches) at the time of his death. This information is based on the "List of Tallest People" page from Wikipedia. There are unverified reports about even taller people in antiquity, which were initially thought to be giant human skeletons but later identified as exaggerated remains of prehistoric animals like whales or elephants.
